### 현대차 구매리뷰

In [ ]:
import requests, html, re

url = "https://www.hyundai.com/wsvc/kr/front/purchaseReview.selectPurchaseReview.do"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/140.0.0.0 Safari/537.36",
    "Accept": "application/json, text/plain, */*",
    "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
    "Referer": "https://www.hyundai.com/kr/ko/purchase-event/vehicles-review"
}

page_size = 50
total_pages = 3

def clean_review_text(text: str) -> str:
    if not text:
        return ""
    t = str(text)
    for _ in range(3):
        t2 = html.unescape(t)
        if t2 == t:
            break
        t = t2
    t = re.sub(r'<\s*br\s*/?>', ' ', t, flags=re.I)
    t = re.sub(r'<[^>]+>', ' ', t)
    t = re.sub(r'[\r\n]+', ' ', t)
    t = re.sub(r'\s+', ' ', t).strip()
    return t

compact_models = ["아반떼", "아반떼 Hybrid", "아반떼 N", "아이오닉 5", "더 뉴 아이오닉 5", "아이오닉 5 N"]
midsize_models = ["쏘나타 디 엣지", "쏘나타 디 엣지 Hybrid", "아이오닉 6"]
large_models = ["그랜저", "그랜저 Hybrid"]
suv_models = [
    "베뉴", "코나", "코나 Hybrid", "코나 Electric", "넥쏘", "디 올뉴 넥쏘",
    "더 뉴 투싼", "더 뉴 투싼 Hybrid", "싼타페", "싼타페 Hybrid", "팰리세이드",
    "디 올 뉴 팰리세이드", "디 올 뉴 팰리세이드 Hybrid", "아이오닉9"
]
van_models = ["스타리아 라운지", "스타리아 라운지 Hybrid", "스타리아", "스타리아 Hybrid", "스타리아 킨더"]
truck_models = ["포터 II Electric", "포터 II Electric 특장차", "ST1"]

gender_map = {"M": "남성", "F": "여성"}
car_type_keys = ['E', 'N', 'P', 'R', 'S']

for car_type_key in car_type_keys:
    for page_no in range(1, total_pages + 1):
        payload = {
            "carType": car_type_key,
            "pageNo": page_no,
            "rowCount": page_size,
            "sortKey": "rating"
        }
        response = requests.post(url, headers=headers, data=payload, timeout=20)
        result_json = response.json()

        if result_json.get("data"):
            reviews = result_json["data"]
            for review in reviews:
                car_name = review.get('carName', '')
                car_name = car_name.replace("더 뉴 아이오닉 5", "더 뉴 아이오닉5")
                car_name = car_name.replace("아이오닉 5", "아이오닉5")
                car_name = car_name.replace("아이오닉 6", "아이오닉6")
                car_name = car_name.replace("아반떼 Hybrid", "아반떼 하이브리드")
                car_name = car_name.replace("쏘나타 디 엣지 Hybrid", "쏘나타 디 엣지 하이브리드")
                car_name = car_name.replace("그랜저 Hybrid", "그랜저 하이브리드")
                car_name = car_name.replace("코나 Hybrid", "코나 하이브리드")
                car_name = car_name.replace("코나 Electric", "코나 일렉트릭")
                car_name = car_name.replace("더 뉴 투싼 Hybrid", "더 뉴 투싼 하이브리드")
                car_name = car_name.replace("싼타페 Hybrid", "싼타페 하이브리드")
                car_name = car_name.replace("디 올 뉴 팰리세이드 Hybrid", "디 올 뉴 팰리세이드 하이브리드")
                car_name = car_name.replace("스타리아 라운지 Hybrid", "스타리아 라운지 하이브리드")
                car_name = car_name.replace("스타리아 Hybrid", "스타리아 하이브리드")

                gender = gender_map.get(review.get("gender"), review.get("gender"))

                if any(model == car_name for model in compact_models):
                    car_type = "준중형차"
                elif any(model == car_name for model in midsize_models):
                    car_type = "중형차"
                elif any(model == car_name for model in large_models):
                    car_type = "대형차"
                elif any(model in car_name for model in suv_models):
                    car_type = 'SUV'
                elif any(model == car_name for model in van_models):
                    car_type = "승합차"
                elif any(model == car_name for model in truck_models):
                    car_type = "화물차"
                else:
                    car_type = ""

                raw_review = review.get("review", "")
                clean_review = clean_review_text(raw_review)
                review_id = review.get("reviewId", "")

                print(f"Id: {review_id}")
                print(f"CarType: {car_type}")
                print("Manufacturer: 현대")
                print(f"Model: {car_name}")
                print(f"Age: {review.get('age')}")
                print(f"Gender: {gender}")
                print(f"Satisfaction: {review.get('carScore')}")
                print(f"Review: {clean_review}\n")
        else:
            print(f"Page {page_no} (carType={car_type_key}) has no data or API error.")


Id: 5113
Manufacturer: 현대
CarType: 
carName: 더 뉴 아이오닉5
age: 30
gender: 남성
carScore: 5
review: 10년 넘게 경유차를 몰다가 처음 전기차로 갈아탔습니다. 처음엔 충전이 귀찮지 않을까, 추울 때 불편하지 않을까 걱정했지만, 이제는 왜 진작 바꾸지 않았나 싶어요. 무엇보다 조용합니다. 시동을 걸어도 소리가 거의 없고, 주행 중에도 실내는 고요합니다. 예전엔 주행 중 진동이나 엔진 소리 때문에 제 강아지가 항상 불안해했는데, 지금은 조수석이나 트렁크에서 편안하게 누워 잠들 정도로 차 안에서 안정감을 느낍니다. 그게 제일 감동이에요. 테슬라도 고민했지만, 아이오닉5는 ‘일상’에 더 잘 녹아드는 차였습니다. 디자인, 공간, 주행 질감, 실내 마감까지 모든 것이 조화롭고, 탄탄한 기본기에 감성까지 갖췄다는 느낌입니다. 전기차답게 정숙하지만, 실내 정돈감과 고급스러움은 테슬라보다 더 높은 만족을 줍니다. 두 차량을 다 시승해보고 고민 없이 선택할 수 있었습니다. 전비 효율도 실속 있습니다. 최근 한 달 평균 전비는 8.8km/kWh, 최고 전비는 11.4km/kWh까지 나왔고, 완속 충전만 이용해도 불편함이 없습니다. 회생제동만 잘 조절해도 출퇴근 100km도 부담 없이 다니고 있어요. 경유차 출퇴근 할 때에 비하면 주유비로 들어가는 돈이 40% 수준으로 떨어졌습니다. 사이버 그레이 외장에 머드 그레이 인테리어, 실물이 훨씬 더 고급스럽습니다. 밤에는 무드등이 실내를 감싸줘서 앉아 있기만 해도 힐링되는 기분이고, 탑승자 모두가 감탄하는 디자인입니다. 그리고 캠핑. 사실 캠핑은 저랑 전혀 상관없는 삶이었습니다. 근데 전기차를 타기 시작하면서 “공조 켜고 조용히 쉴 수 있다”는 게 얼마나 큰 장점인지 알게 됐고, 지금은 주말 캠핑이 기다려질 정도입니다. 요즘에는 이것저것 캠핑용품을 하나씩 사는 재미에 빠졌습니다. 이 차는 이동수단이 아니라, ‘새로운 라이프스타일’을 시작하게 해준 계기가 됐습니다. 탈수록 

### 캐스퍼 구매리뷰

In [1]:
import requests, re, html
def clean_text(text: str):
    if text is None:
        return None
    t = html.unescape(str(text))
    t = re.sub(r'<\s*br\s*/?>', ' ', t, flags=re.I) 
    t = re.sub(r'[\r\n]+', ' ', t)                  
    t = re.sub(r'\s+', ' ', t).strip()             
    return t

url = "https://casper.hyundai.com/gw/wp/product/v2/product/epilogues"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
    "Accept": "application/json, text/plain, */*"
}

page_size = 100
total_pages = 3

for page_no in range(1, total_pages + 1):
    params = {
        "siteTypeCode": "W",
        "carCode": ['AX01','AX02'],
        "enginCode": "",
        "trimCode": "",
        "orderType": "2",
        "pageNo": page_no,
        "pageSize": page_size,
        "screenCode": "B"
    }

    response = requests.get(url, headers=headers, params=params)
    result_json = response.json()

    if result_json.get("data") and result_json["data"].get("list"):
        for item in result_json["data"]["list"]:
            Id = item.get("epilogueNumber")
            CarType = "경차"
            Manufacturer = "현대"
            Model = item.get("carName")
            Age = item.get("customerAgeSectionCode")
            Gender = item.get("customerGenderName")
            Satisfaction = item.get("satisfactionScore")
            Review = clean_text(item.get("epilogueContents"))

            print(f"Id: {Id}")
            print(f"CarType: {CarType}")
            print(f"Manufacturer: {Manufacturer}")
            print(f"Model: {Model}")
            print(f"Age: {Age}")
            print(f"Gender: {Gender}")
            print(f"Satisfaction: {Satisfaction}")
            print(f"Review: {Review}\n")
    else:
        print(f"Page {page_no} has no data or API error.")


Id: 11680
CarType: 경차
Manufacturer: 현대
Model: 캐스퍼
Age: 50
Gender: 여성
Satisfaction: 5
Review: 처음 캐스퍼 출시 때부터 눈여겨보던 차입니다! 직장 이직으로 출퇴근이 어려워 드디어 구매 하게 되었네요^^ 남들은 그 돈이면 더 좋은 차를 살 수 있다고 하지만 저는 풀옵션을 포기할 수가 없었어요! 또 한 경차 혜택도 무시 못하구요~^^ 온라인 구매라 어려움이 있을 줄 알았는데, 생각보다 어렵지 않았습니다! 문자로 친절히 알려주기도 하고 등록 대행 서비스를 이용하니 금방금방되더라구요! 아직 운전이 미숙해서 연습을 많이 해야 하지만 경차 치고 좋은 기능들이 많아서 금방 배울수있을거같습니다^^ 고민하는 시간이 너무 아까워서 저는 고민 할 시간도 없이 빨리 결정했는데 너무 너무 만족스러워요! 승차감도 저는 괜찮았고, 터보라 그런지 차도 잘 나가는 기분이 들었습니다! 또 출퇴근용으로 구입했지만 캠핑용으로도 좋다고해서 캠핑도 자주 다닐 생각입니다^^ 그럼 모두 안전운행하시고! 좋은하루되세요~~

Id: 11877
CarType: 경차
Manufacturer: 현대
Model: 캐스퍼
Age: 30
Gender: 여성
Satisfaction: 5
Review: 경차의 장점은 다들 아시지만 캐스퍼는 현재 경차시장 중 제일이죠!!!ㅎㅎ 제가 사는 지역에는 시승 가능한 곳이 2시간은 이동해야 가능해서 사실 시승도 못해보고 경차의 장점과 경차별 성능만 대비 해 보고 구매한 캐스퍼!! 시승 못해본 불안감은 차를 딱! 타고 운전 해 보는 순간 날아갔어요~ 혹시 저처럼 지방에 살아 캐스퍼 시승이 힘들어도 걱정하시지 않으셔도 될 듯 합니다!! 거기다가 4월 진행되는 경차 저금리 할부 이벤트로 구매했더니 1.9%로 할부 진행하여서 너무 좋아요 :) 4월 뿐 아니라 5월 6월 이벤트가 있을 지 모르겠지만 구매자에게 많은 힘이 됩니다!ㅎㅎㅎ 경차가 튼튼하면 얼마나 튼튼하겠냐고 안정성에서 고민하시는 분들 중 경제적인 면에서 어차